In [19]:
ptype = _G.type

In [20]:
type = function(x)
    local pt = ptype(x)
    return pt == "table" and x.__datatype or pt
end

In [6]:
type{__datatype="Keys"}

"Keys"

In [2]:
-- local typesep, mm

-- local db = require"util.Debug"
typesep = ";"

mm = function(gen)
  local f = {gen}
  f.addmethod = function(self, ...)
    local types = ""
    local nargs = select("#", ...)
    for i = 1, nargs - 2 do
      types = types .. select(i, ...) .. typesep
    end
    types = types .. select(nargs - 1, ...)
    self[types] = select(nargs, ...)
    return self
  end
  f.setgeneric = function(self, m) self[1] = m end
  f.getgeneric = function(self) return self[1] end
  f.getmethod = function(...)
    local types = ""
    local nargs = select("#", ...)
    local method = f[1]
    for i = 1, nargs do
      types = types .. type(select(i, ...))
      if f[types] then
        method = f[types]
      end
      types =  types .. typesep
    end
    return method
  end
  return setmetatable(f,{__call = function(self, ...)
    local method = f.getmethod(...)
    if not method then error("no method found") end
    return method(...)
  end})
end

return mm


<function 1>

In [9]:
f = mm()
f:addmethod("number", "number", function(x, y,...) return x * y*... end)
f:addmethod("string", "number", function(x, y) return x .. tostring(y) end)
f:addmethod("Keys", function(x, y) return x end)
for k, v in pairs(f) do
    print(k, v)
end
return f

getgeneric	function: 0x563bf45346f0
addmethod	function: 0x563bf45246a0
number;number	function: 0x563bf4537bc0
getmethod	function: 0x563bf4548910
setgeneric	function: 0x563bf45263b0
Keys	function: 0x563bf4543480
string;number	function: 0x563bf4543780


{Keys = <function 1>,addmethod = <function 2>,getgeneric = <function 3>,getmethod = <function 4>,["number;number"] = <function 5>,setgeneric = <function 6>,["string;number"] = <function 7>,<metatable> = {__call = <function 8>}}

In [11]:
f("2qe",2,3,4)

"2qe2"

In [14]:
local f
m = {__datatype = "Module"}
f = mm()
f:setgeneric(function(...) return vmap(type, ...) end)
f:addmethod("Module", "number", function(x, y) return "Module called" end)
m.f = f
return m.f

{ <function 1>,["Module;number"] = <function 2>,addmethod = <function 3>,getgeneric = <function 4>,getmethod = <function 5>,setgeneric = <function 6>,<metatable> = {__call = <function 7>}}

In [22]:
m:f(1)

"Module called"

In [16]:
vmap = function(f, ...)
    local ret = {}
    for i = 1, select("#", ...) do
        ret[i] = f(select(i, ...))
    end
    return table.unpack(ret)
end